+ 整理 PKG bincode 的編碼
    + 編碼 by 料號切割 CIE / IV / VF  , CIE+WL , 
    + 庫存料號編碼 

In [1]:
#Load DB 元件
import os
os.chdir("D://AP//PythonLen//PKG_SPEC_TurnRate//")
import ConnectDB as cn
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime
print(os.path.abspath('.'))

D:\AP\PythonLen\PKG_SPEC_TurnRate


In [2]:
# bincode 編碼定義
def GetBinNaming():
    sSQL=" SELECT [SN],[Model_Type],[Naming_Rule_Number],[Naming_Rule],[Spec_Type],[code_index],[code_length] "
    sSQL+=" ,[Cie_Pos_Count] FROM [Planning].[dbo].[C_BIN_Naming] "  
    return sSQL

In [3]:
# 料號對應編碼
def GetRuleNumber():
    sSQL=" SELECT MODEL_NAME,PART_NO,VERSION,ISMODIFY,NAMING_RULE_NUMBER FROM R_ENG_SPEC_VER "
    return sSQL

In [4]:
### 目前庫存取 F/G
#oh_type=1 , onhand
def GetNOH():
    sSQL=" SELECT INV.PRODUCTNO, INV.BINCODE, SUM(INV.QTY) OH_QTY "
    sSQL+="   FROM WHMSRPT.TBLINVFGDINVENTORY INV "
    sSQL+="  WHERE INV.PRODUCTNO LIKE '95%' "
    sSQL+="    AND INV.SOURCE = 1 "
    sSQL+="    AND INV.INVENTORYNO LIKE 'F/G%' "
    sSQL+="  GROUP BY INV.PRODUCTNO, INV.BINCODE "
    return sSQL

In [5]:
# Model Group & Model Name
def ModelProp():
    sql="SELECT distinct l.model_group,l.model_name,l.part_no FROM erprpt.vsinv_itf_mtl l  where l.part_no like '95%' "  
    return sql

In [6]:
#目前 WHMS 庫存
strSql=GetNOH()
dbCon=cn.OracCn()
dtNOH_T05=dbCon.Exec_OracleCommend(dbCon.T05HIS01,strSql)
dtNOH_C01=dbCon.Exec_OracleCommend(dbCon.C01HIS01,strSql)
dtNOH_S01=dbCon.Exec_OracleCommend(dbCon.S01HIS01,strSql)
dtNOH_C01=pd.merge(dtNOH_C01,dtNOH_S01,how='outer')
dtNOH=pd.merge(dtNOH_C01,dtNOH_T05,how='outer')
del dtNOH_T05,dtNOH_C01,dtNOH_S01

In [7]:
dtNOH.head(2)

PRODUCTNO   BINCODE  OH_QTY
0  95.S3806.W0A0C8N  AEF0XQKd   43000
1  95.S3806.W0A0C8N  BDG0XQJd   71000

In [8]:
# Model , 料號
strSql=ModelProp()
dbCon=cn.OracCn()
dtModelPro=dbCon.Exec_OracleCommend(dbCon.T01HIS1,strSql)

In [9]:
dtModelPro.head(2)

MODEL_GROUP MODEL_NAME           PART_NO
0      303007  PC33X50.0  95.C3030.XTC005N
1      303006  PC33H42.1  95.C3030.HBC002Z

In [10]:
dtNOH=pd.merge(dtNOH,dtModelPro,left_on='PRODUCTNO',right_on='PART_NO',how='left')

In [12]:
del dtModelPro
dtNOH=dtNOH.drop('PART_NO',axis=1)
dtNOH.head(2)

PRODUCTNO   BINCODE  OH_QTY MODEL_GROUP MODEL_NAME           PART_NO
0  95.S3806.W0A0C8N  AEF0XQKd   43000      380610  PS06W12.3  95.S3806.W0A0C8N
1  95.S3806.W0A0C8N  BDG0XQJd   71000      380610  PS06W12.3  95.S3806.W0A0C8N

In [32]:
strSql=GetRuleNumber()
dtPrdRule=dbCon.ExecMSSql_Commend(dbCon.lexmsdb2,'Planning',strSql)
dtPrdRule.head(2)

MODEL_NAME PART_NO  VERSION ISMODIFY  NAMING_RULE_NUMBER
0  PT30A02.0       *        0        Y                 5.0
1  PT30A02.0       *        1        N                 5.0

In [28]:
# Model, Part 對應料號
dtPrdRule.head(2)
NAMING_RULE=dtPrdRule.groupby(['MODEL_NAME','PART_NO','NAMING_RULE_NUMBER'])

In [37]:
NAMING_RULE.agg(['count','max'])

VERSION     ISMODIFY    
                                                      count max    count max
MODEL_NAME   PART_NO             NAMING_RULE_NUMBER                         
00.95000.264 00.95000.264        17.0                     2   1        2   Y
None.0       00.50000.028        31.0                     2   1        2   Y
             00.50000.029        31.0                     1   0        1   Y
             00.95000.619        7.0                      1   0        1   Y
             00.95000.622        31.0                     2   1        2   Y
             00.95000.634        7.0                      1   0        1   Y
             00.95000.733        7.0                      1   0        1   Y
             00.95000.778        31.0                     1   0        1   Y
             00.95000.779        31.0                     1   0        1   Y
             00.95000.807        32.0                     1   0        1   Y
             00.95000.816        6.0                      4   3        4   Y
                                 35.0                     4   7        4   Y
             00.95000.823        7.0                      1   0        1   Y
             00.95000.836        0.0                      1   1        1   N
                                 3.0                      2   2        2   Y
             00.95000.83A        24.0                     2   1        2   Y
             00.95000.83W        11.0                     1   0        1   Y
             00.95000.842        32.0                     1   0        1   Y
             00.95000.848        6.0                      2   1        2   Y
             00.95000.84A        32.0                     1   0        1   Y
             00.95000.84N        40.0                     1   0        1   Y
             00.95000.851        32.0                     1   0        1   Y
             00.95000.86M        32.0                     1   0        1   Y
             00.95000.87G        32.0                     3   2        3   Y
             00.95000.87S        32.0                     1   0        1   Y
             00.95000.88H        32.0                     1   0        1   Y
             95.C3000.0C0        3.0                      1   0        1   Y
             95.C3528.HAB000X    31.0                     1   0        1   Y
             95.C3528.HAC009X    31.0                     1   0        1   Y
             95.C3528.HBB000X    31.0                     1   0        1   Y
...                                                     ...  ..      ...  ..
PU55S01.0    95.U5050.WSB100Z    35.0                     1   0        1   Y
             95.U5050.XHA100Z    35.0                     1   0        1   Y
PU88S01.0    50.U3535.XVA101Z    35.0                     1   0        1   Y
             95.E3535.PT1W00Z    35.0                     1   0        1   Y
             95.U3535.XVA100Z    0.0                      2   4        2   N
                                 35.0                     3   3        3   Y
             95.U3535.XVA101Z    35.0                     1   0        1   Y
             95.U3535.YVA100Z    0.0                      2   4        2   N
                                 35.0                     3   3        3   Y
             95.U3535.ZVA100Z    0.0                      2   4        2   N
                                 35.0                     3   3        3   Y
PU88S04.0    95.U3535.WFA100Z    0.0                      1   4        1   N
                                 35.0                     4   3        4   Y
             95.U3535.WFA101Z    0.0                      3   4        3   N
                                 35.0                     2   2        2   Y
PU88S11.0    95.U3535.ZVB100Z    35.0                     6   5        6   Y
PU88S31.0    95.U3535.VFD100Z    35.0                     1   0        1   Y
PUA1S01.0    95.E3535.PT0100Z    17.0                     1   0        1   Y
PUB3S01.0    95.U5630.XFB100Z    0.0            

In [24]:
# 編碼規則,
strSql=GetBinNaming()
dbCon=cn.OracCn()
dtBinEncoder=dbCon.ExecMSSql_Commend(dbCon.lexmsdb2,'Planning',strSql)
dtBinEncoder.head(2)

SN Model_Type  Naming_Rule_Number Naming_Rule Spec_Type  code_index  \
0   1    Package                   1         COB       CIE           1   
1   2    Package                   1         COB       CRI           4   

   code_length  Cie_Pos_Count  
0            3              4  
1            1              4

In [ ]:
Naming_Rule=dtBinRule.groupby('Naming_Rule').size()

In [ ]:
Naming_Rule.sort_values(ascending=False)

In [ ]:
Naming_Rule=dtBinRule.groupby('Spec_Type').size()

In [ ]:
Naming_Rule